In [ ]:
import pytest
from unittest.mock import AsyncMock, Mock, patch
import httpx
from fh_saas.utils_api import AsyncAPIClient
from fh_saas.utils_graphql import GraphQLClient

## Test GraphQL Query Execution

In [ ]:
@pytest.mark.asyncio
async def test_execute_query_success():
    """Test successful GraphQL query execution"""
    
    # Mock response
    mock_response = Mock(spec=httpx.Response)
    mock_response.status_code = 200
    mock_response.json.return_value = {
        'data': {
            'users': [{'id': 1, 'name': 'Alice'}]
        }
    }
    mock_response.raise_for_status = Mock()
    
    # Mock API client
    with patch('httpx.AsyncClient') as MockClient:
        mock_client_instance = AsyncMock()
        mock_client_instance.request.return_value = mock_response
        MockClient.return_value = mock_client_instance
        
        async with AsyncAPIClient('https://api.example.com/graphql') as api_client:
            client = GraphQLClient(api_client)
            
            result = await client.execute_query(
                query='{ users { id name } }'
            )
            
            assert result == {'data': {'users': [{'id': 1, 'name': 'Alice'}]}}

# Run test
await test_execute_query_success()
print("✅ Test passed: Execute query success")

✅ Test passed: Execute query success


In [ ]:
@pytest.mark.asyncio
async def test_execute_query_with_errors():
    """Test GraphQL query with errors in response"""
    
    # Mock response with errors
    mock_response = Mock(spec=httpx.Response)
    mock_response.status_code = 200
    mock_response.json.return_value = {
        'errors': [{'message': 'Field not found'}]
    }
    mock_response.raise_for_status = Mock()
    
    with patch('httpx.AsyncClient') as MockClient:
        mock_client_instance = AsyncMock()
        mock_client_instance.request.return_value = mock_response
        MockClient.return_value = mock_client_instance
        
        async with AsyncAPIClient('https://api.example.com/graphql') as api_client:
            client = GraphQLClient(api_client)
            
            try:
                await client.execute_query('{ invalid }')
                assert False, "Should have raised ValueError"
            except ValueError as e:
                assert 'Field not found' in str(e)

# Run test
await test_execute_query_with_errors()
print("✅ Test passed: Execute query with errors")

✅ Test passed: Execute query with errors


## Test Pagination Generator

In [ ]:
@pytest.mark.asyncio
async def test_fetch_pages_generator_two_pages():
    """Test generator yields exactly 2 batches for 2-page dataset"""
    
    # Mock responses for 2 pages
    page1_response = Mock(spec=httpx.Response)
    page1_response.status_code = 200
    page1_response.json.return_value = {
        'data': {
            'users': {
                'nodes': [{'id': 1}, {'id': 2}],
                'pageInfo': {'hasNextPage': True, 'endCursor': 'cursor2'}
            }
        }
    }
    page1_response.raise_for_status = Mock()
    
    page2_response = Mock(spec=httpx.Response)
    page2_response.status_code = 200
    page2_response.json.return_value = {
        'data': {
            'users': {
                'nodes': [{'id': 3}, {'id': 4}],
                'pageInfo': {'hasNextPage': False, 'endCursor': None}
            }
        }
    }
    page2_response.raise_for_status = Mock()
    
    with patch('httpx.AsyncClient') as MockClient:
        mock_client_instance = AsyncMock()
        # Return different responses for each call
        mock_client_instance.request.side_effect = [page1_response, page2_response]
        MockClient.return_value = mock_client_instance
        
        async with AsyncAPIClient('https://api.example.com/graphql') as api_client:
            client = GraphQLClient(api_client)
            
            batches = []
            async for batch in client.fetch_pages_generator(
                query_template='''
                    query($cursor: String) {
                        users(after: $cursor) {
                            nodes { id }
                            pageInfo { hasNextPage endCursor }
                        }
                    }
                ''',
                variables={'cursor': None},
                items_path=['data', 'users', 'nodes'],
                cursor_path=['data', 'users', 'pageInfo', 'endCursor'],
                has_next_path=['data', 'users', 'pageInfo', 'hasNextPage']
            ):
                batches.append(batch)
            
            # Verify exactly 2 batches
            assert len(batches) == 2
            assert batches[0] == [{'id': 1}, {'id': 2}]
            assert batches[1] == [{'id': 3}, {'id': 4}]

# Run test
await test_fetch_pages_generator_two_pages()
print("✅ Test passed: Fetch pages generator (2 pages)")

✅ Test passed: Fetch pages generator (2 pages)


In [ ]:
@pytest.mark.asyncio
async def test_cursor_extraction_and_update():
    """Test cursor is correctly extracted and updated between pages"""
    
    # Mock responses
    page1_response = Mock(spec=httpx.Response)
    page1_response.status_code = 200
    page1_response.json.return_value = {
        'data': {
            'users': {
                'nodes': [{'id': 1}],
                'pageInfo': {'endCursor': 'CURSOR_ABC'}
            }
        }
    }
    page1_response.raise_for_status = Mock()
    
    page2_response = Mock(spec=httpx.Response)
    page2_response.status_code = 200
    page2_response.json.return_value = {
        'data': {
            'users': {
                'nodes': [{'id': 2}],
                'pageInfo': {'endCursor': None}  # Last page
            }
        }
    }
    page2_response.raise_for_status = Mock()
    
    with patch('httpx.AsyncClient') as MockClient:
        mock_client_instance = AsyncMock()
        mock_client_instance.request.side_effect = [page1_response, page2_response]
        MockClient.return_value = mock_client_instance
        
        async with AsyncAPIClient('https://api.example.com/graphql') as api_client:
            client = GraphQLClient(api_client)
            
            variables = {'cursor': None}
            batch_count = 0
            
            async for batch in client.fetch_pages_generator(
                query_template='query($cursor: String) { users(after: $cursor) { nodes { id } pageInfo { endCursor } } }',
                variables=variables,
                items_path=['data', 'users', 'nodes'],
                cursor_path=['data', 'users', 'pageInfo', 'endCursor']
            ):
                batch_count += 1
            
            # Verify 2 batches were processed
            assert batch_count == 2
            
            # Verify the second request was made with the extracted cursor
            assert mock_client_instance.request.call_count == 2
            
            # Check the variables in the second call included the cursor
            second_call_kwargs = mock_client_instance.request.call_args_list[1][1]
            second_call_json = second_call_kwargs.get('json', {})
            second_call_variables = second_call_json.get('variables', {})
            
            assert second_call_variables.get('cursor') == 'CURSOR_ABC', \
                f"Expected cursor 'CURSOR_ABC', got {second_call_variables.get('cursor')}"

# Run test
await test_cursor_extraction_and_update()
print("✅ Test passed: Cursor extraction and update")

✅ Test passed: Cursor extraction and update
